# **Machine Learning - Proyecto en Clase: Games**

## Objetivo

El objetivo principal de ese código de notebook es construir, entrenar y guardar un modelo de Machine Learning (Regresión) capaz de predecir las ventas totales (`total_sales`) de un videojuego basándose en sus características (plataforma, género, año, puntuaciones, etc.).

## Librerías

### Librerías generales

In [50]:
# importar librerías generales
import pandas as pd
import numpy as np
import time

### Librerías de ML

In [ ]:
# importar funciones de ML
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
#from lightgbm import LGBMRegressor
#from xgboost import XGBRegressor
#from sklearn.model_selection import GridSearchCV

## Cargar datos limpios

Leer el archivo `games_clean.csv` que fue el resultado del pipeline de limpieza anterior.

In [52]:
ruta = r"C:\Users\fnaje\OneDrive\Documents\UniAndes\2do Seminario\ejemplo-proyecto-demo-games\data\processed\games_clean.csv"

In [53]:
games_clean = pd.read_csv(ruta)

In [54]:
games_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8296 entries, 0 to 8295
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   videogame_names            8296 non-null   object 
 1   platform                   8296 non-null   object 
 2   year_of_release            8296 non-null   int64  
 3   genre                      8296 non-null   object 
 4   na_sales                   8296 non-null   float64
 5   eu_sales                   8296 non-null   float64
 6   jp_sales                   8296 non-null   float64
 7   other_sales                8296 non-null   float64
 8   critic_score               8296 non-null   float64
 9   user_score                 8296 non-null   float64
 10  rating_esrb                8296 non-null   object 
 11  total_sales                8296 non-null   float64
 12  gen_platform               8296 non-null   object 
 13  classification_user_score  8296 non-null   objec

# Random Forest

## Preparar los datos para el modelo

### Seleccionar variable objetivo y variables dependientes

Los pasos son los siguientes: 
- Seleccionar las columnas de entrada (`features` o `X`) que se usará para predecir
    - `platform`, `genre`, `critic_score`, `user_score`, `year_of_release`
- Seleccionar la columna objetivo (`target` o `y`) que se quiere predecir
    - `total_sales`
- Convertir las columnas categóricas en un formato numérico para que el modelo entienda. 
    - `One-Hot Ecoding`. 

In [55]:
# seleccionar las columnas categóricas y numéricas
col_categoricas = ['platform', 'genre', 'rating_esrb']
col_numericas = ['year_of_release', 'critic_score', 'user_score']

In [56]:
# seleccionar columna objetivo
target = 'total_sales'

In [57]:
# separar X e y
X_categoricas = games_clean[col_categoricas]
X_numericas = games_clean[col_numericas]
y = games_clean[target]

In [58]:
X_categoricas

,platform,genre,rating_esrb
0,Wii,Sports,E
1,Wii,Racing,E
2,Wii,Sports,E
3,DS,Platform,E
4,Wii,Misc,E
...,...,...,...
8291,PC,Action,M
8292,PC,Shooter,T
8293,PC,Strategy,E10+
8294,PC,Adventure,RP


In [59]:
X_numericas

,year_of_release,critic_score,user_score
0,2006,76.0,8.0
1,2008,82.0,8.3
2,2009,80.0,8.0
3,2006,89.0,8.5
4,2006,58.0,6.6
...,...,...,...
8291,2014,80.0,7.6
8292,2011,61.0,5.8
8293,2011,60.0,7.2
8294,2009,63.0,5.8


In [60]:
y

0       82.54
1       35.52
2       32.77
3       29.80
4       28.91
        ...  
8291     0.01
8292     0.01
8293     0.01
8294     0.01
8295     0.01
Name: total_sales, Length: 8296, dtype: float64

## Aplicación One-Hot Encoding

Vamos a aplicar una función que nos ayudará a convertir todas las variables categócias en números. 

El One-Hot Encoding (codificación en un solo punto) es una técnica de preprocesamiento de datos utilizada para convertir variables categóricas en un formato numérico que los algoritmos de aprendizaje automático puedan entender y procesar. 

La mayoría de los modelos de machine learning requieren que los datos de entrada sean números, por lo que esta técnica es esencial.

Para esto vamos a utilizar la función `OneHotEncoder` y uno de los argumentos que tiene y que vamos a utilizar es: 

`sparse_output=False dense array`
- Formato de salidad que produce el `OneHotEncoder` después de transformar los datos. 
- `sparse_output=True` devuelve una matriz despersa, en caso de tener muchísimas categorías (cientos o miles), ya que guarda la ubicación de los "1" y asume que todo lo demás es "0". 
    - Desventaja: las matrices dispersas son más difíciles de inspeccionar y combinar directamente con DF de Pandas. 
- `sparse_output=False` indicar que devuelva una matriz densa (dense array) normal de NumPy. La matriz contendrá explícitamente todos los ceros y unos. 
- handel_unknown=`ignore`: se utiliza para gestionar categorías que aparecen en el conjunto de datos de prueba pero no estaban preesentes en el conjunto de entrenamiento. 

In [61]:
# función OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder

,categories,'auto'
,drop,None
,sparse_output,False
,dtype,<class 'numpy.float64'>
,handle_unknown,'ignore'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


De ahí con la función `OneHotEncoder` vamos a aplicar el médotod `fit_transform()`

¿Qué hace `fit_transform()`?

El método `.fit_transform()` viene de la librería `scikit-learn`. Todos los "transformadores" de `scikit-learn` (como `OneHotEncoder`, `StandardScaler`, `TfidfVectorizer`, etc.) tienen este método.

El método combina dos pasos en uno para mayor conveniencia y eficiencia. 
- `fit()`: ajusta, es decir, analiza los datos de entrada para aprender los parámetros de la transformación. 
    - Identifica todas las categorías unicas en cada columna. 
    - Prepara una estructura de maeo para crear las nuevas columnas binarias. 
- `transforma()`: transforma, una vez que el encoder ha aprendido qué categorías existen, este paso aplica la transformación a los datos. 
    - Crea nuevas columnas binarias para cada categoría única. 
    - Asigna valores 0 y 1 en las filas correspondientes. 

In [62]:
# aplicar One-Hot Encoding
X_categoricas_encoded = encoder.fit_transform(X_categoricas)

In [63]:
X_categoricas_encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], shape=(8296, 43))

Ahora con estos datos, podemos aplicar el siguiente método: 

`.get_feature_names_out()` método de OneHotEncoder que se utiliza para obtener los nombres de las nuevas columnas creadas después de aplicar el one-hot encoding. 

In [64]:
# nombres de columnas del onehotencoder
encoder.get_feature_names_out(col_categoricas)

array(['platform_2600', 'platform_3DS', 'platform_DC', 'platform_DS',
       'platform_GB', 'platform_GBA', 'platform_GC', 'platform_GEN',
       'platform_N64', 'platform_NES', 'platform_PC', 'platform_PS',
       'platform_PS2', 'platform_PS3', 'platform_PS4', 'platform_PSP',
       'platform_PSV', 'platform_SAT', 'platform_SNES', 'platform_Wii',
       'platform_WiiU', 'platform_X360', 'platform_XB', 'platform_XOne',
       'genre_Action', 'genre_Adventure', 'genre_Fighting', 'genre_Misc',
       'genre_Platform', 'genre_Puzzle', 'genre_Racing',
       'genre_Role-Playing', 'genre_Shooter', 'genre_Simulation',
       'genre_Sports', 'genre_Strategy', 'rating_esrb_AO',
       'rating_esrb_E', 'rating_esrb_E10+', 'rating_esrb_K-A',
       'rating_esrb_M', 'rating_esrb_RP', 'rating_esrb_T'], dtype=object)

In [65]:
# crear nuevo df con las nuevas columnas codificadas
games_encoded = pd.DataFrame(
    X_categoricas_encoded, 
    columns=encoder.get_feature_names_out(col_categoricas)
)

In [66]:
# visualización de datos
print(f"Filas x columnas de variables categóricas codificadas: {games_encoded.shape}")
print()
display(games_encoded.head())

Filas x columnas de variables categóricas codificadas: (8296, 43)



,platform_2600,platform_3DS,platform_DC,platform_DS,platform_GB,platform_GBA,platform_GC,platform_GEN,platform_N64,platform_NES,...,genre_Simulation,genre_Sports,genre_Strategy,rating_esrb_AO,rating_esrb_E,rating_esrb_E10+,rating_esrb_K-A,rating_esrb_M,rating_esrb_RP,rating_esrb_T
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Ahora que tenemos un df de nuevas categorías con números, podemos unir las columnas numéricas con las categóricas

In [67]:
# unión columnas numéricas con categóricas codificadas
X = pd.concat([X_numericas.reset_index(drop=True), games_encoded], axis=1)

In [68]:
print(f"Shape final de Features (X): {X.shape}")
print("Primeras filas de X final:")
display(X.head())

Shape final de Features (X): (8296, 46)
Primeras filas de X final:


,year_of_release,critic_score,user_score,platform_2600,platform_3DS,platform_DC,platform_DS,platform_GB,platform_GBA,platform_GC,...,genre_Simulation,genre_Sports,genre_Strategy,rating_esrb_AO,rating_esrb_E,rating_esrb_E10+,rating_esrb_K-A,rating_esrb_M,rating_esrb_RP,rating_esrb_T
0,2006,76.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2008,82.0,8.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2009,80.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2006,89.0,8.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2006,58.0,6.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## Dividir los datos

Listo, ya podemos dividir los datos en conjuntos de entrenamiento (`train`) y prueba (`test`). Esta es una práctica fundamental en el aprendizaje automático. El objetivo es evaluar de manera objetiva el rendimiento de un modelo en datos que nunca ha visto antes, garantizando que el modelo sea capaz de generalizar su conocimiento a nuevos datos, en lugar de simplemente memorizar los datos de entrenamiento. 

Razones clave para utilizar `train_test_split`

1. Prevenir el sobreajuste (`overfitting`)
- ¿Qué es? 
    - El sobreajuste ocurre cuando un modelo aprende el conjunto de datos de entrenamiento con demasiado detalle, incluyendo el ruido y las peculiaridades que no son representativas del conjunto de datos completo. 
    - El resultado es que el modelo funciona excepcionalmente bien en los datos de entrenamiento, pero muy mal en datos nuevos y desconocidos.
- ¿Cómo ayuda la división? 
    - Al reservar una parte de los datos como conjunto de prueba, puedes entrenar el modelo en el conjunto de entrenamiento y luego evaluar su rendimiento en el conjunto de prueba. 
    - Si el rendimiento en el conjunto de entrenamiento es muy alto pero en el conjunto de prueba es bajo, es un claro indicador de que el modelo está sobreajustado. 
2. Medir la capacidad de generalización 
 - ¿Qué es?
    - La generalización es la capacidad de un modelo para hacer predicciones precisas en datos que no ha visto durante el entrenamiento. Un modelo que generaliza bien ha aprendido los patrones subyacentes de los datos, no los datos específicos.
- ¿Cómo ayuda la división? 
    - El conjunto de prueba actúa como un sustituto de los datos del mundo real. Al evaluar el modelo en este conjunto de datos retenido, se obtiene una estimación realista de cómo se comportará el modelo en la práctica. 
3. Evaluar el modelo de forma imparcial
    - Si se entrena y se evalúa el modelo con los mismos datos, los resultados de la evaluación no serán confiables. El modelo ya ha "visto las respuestas" y podría estar memorizando en lugar de aprendiendo.
    - El conjunto de prueba proporciona una métrica de rendimiento imparcial, ya que el modelo se prueba en datos que son completamente nuevos para él. 
4. Asegurar la reproducibilidad (`random_state`) 
    - ¿Qué es? 
        - La división de los datos es un proceso aleatorio. Sin `random_state`, cada vez que ejecutes el código, la división será diferente, lo que significa que el rendimiento del modelo también podría variar ligeramente.
    - ¿Cómo ayuda `random_state` con un valor fijo? 
        - Al establecer una "semilla" para el generador de números aleatorios (`random_state`), se asegura que la división de datos sea exactamente la misma cada vez que se ejecute el código. 
        - Esto es crucial para la reproducibilidad, permitiendo comparar los resultados de manera consistente.

In [69]:
# variables definidas para usar más adelante
RANDOM_STATE = 50
TEST_SIZE = 0.2

In [70]:
# división datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

In [71]:
print(f"Tamaño X_train: {X_train.shape}")
print(f"Tamaño X_test: {X_test.shape}")
print(f"Tamaño y_train: {y_train.shape}")
print(f"Tamaño y_test: {y_test.shape}")

Tamaño X_train: (6636, 46)
Tamaño X_test: (1660, 46)
Tamaño y_train: (6636,)
Tamaño y_test: (1660,)


## Entrenar el modelo (Random Forest Regressor)

`RandomForestRegressor` es un algoritmo de aprendizaje automático que forma part de la familia de métodos de aprendizaje en conjunto (ensemble learning). En lugar de utilizar un sólo modelo para predecir, combina las predicciones de múltiples árboles de decisión para objetner un resultado más preciso y robusto. 

Se utiliza específicamente para tareas de regresión, es decir, predecir un valor numérico continuo, como precios, ventas, temperatura, etc. 

¿Qué es un árbol de decisión?

Algoritmo de aprendizaje supervisado que utiliza una estructura jerárquica similar a un diagrama de flujo para tomar decisiones o hacer predicciones. 

Sirve para problemas de clasificación (predicción de categorías) y regresión (predicción de valores numéricos). 

Aquí están los argumentos y sus usos: 
- `n_estimators`: determina el número de árboles de decisión que el modelo construirá en el bostque. Cada árbol se entrena en una submuestra aleatoria de los datos. La predicción final se basa en el promedio de las predicciones de todos los árboles. Un buen punto de partida es 100
- `random_state`: controla la aleatoriedad en el proceso de creación de árboles.
- `n_jobs`: indica al algoritmo cuántos núcleos del CPU puede usar para ejecutar las tareas en paralelo. Con un valor de -1 se indica que use todos los núcleos de CPU disponibles en la máquina, maximiza la velocidad de entrenamiento. 
- `oob_score`: habilita la evaluación "out-of-bag"
    - Cuando se crea un Random Forest, el algoritmo construye muchos árboles de decisión (`n_estimators=100`). Cada uno de estos árboles no se entrena con todos los datos de `X_train`. En su lugar, cada árbol se entrena con una muestra aleatoria con reemplazo (llamada `bootstrap sample`).
    - Esto significa que pra cada árbol individual, habrá algunas filas de `X_Train` que no se usaron para entrenarlo. Esas filas se llama `Out-of-Bag`. 
    - Al poner `True` le estamos diciendo mientras te entrenas, para cada fila en `X_train`, usa solo los árboles que NO vieron esa fila durante su entrenamiento (sus muestras OOB) para hacer una predicción sobre ella. Luego, compara todas estas predicciones OOB con los valores reales (`y_train`) y calcula una métrica de rendimiento general.

In [72]:
# creamos el modelo RandomForestRegressor con configuraciones específicas
modelo = RandomForestRegressor(
    n_estimators=100,      # ¿cuántos árboles de decisión construir? 100 es un buen punto de partida
    random_state=RANDOM_STATE, # fija la aleatoriedad para que los resultados sean reproducibles
    n_jobs=-1,             # usa todos los núcleos de CPU disponibles para entrenar más rápido
    oob_score=True         # pide al modelo que calcule un score "Out-of-Bag"
)

Una vez puesto el modelo, hay el método `fit()` que entrena el modelo, para permitirle aprender la relación entre las característica de entrada (`X_train`) y la variable objetivo (`y_train`)

In [73]:
# entrenar el modelo
modelo.fit(X_train, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,True


¿Qué métrica se calcula aquí?

Para `RandomForestRegressor`, la métrica que calcula el `oob_score` es el Coeficiente de Determinación (`R²`).

- El R² mide qué porcentaje de la varianza de la variable objetivo (`total_sales`) es explicado por el modelo.
- Va de -∞ a 1.0
    - 1.0: el modelo explica perfectamente toda la variabilidad (ideal).
    - 0.0: el modelo no explica nada mejor que simplemente predecir la media.
    - Negativo: el modelo es peor que predecir la media (muy malo).

In [74]:
# mostrar el OOB Score (una estimación del rendimiento sin usar el test set)
print(f"OOB Score (R^2 estimado): {modelo.oob_score_:.4f}")

OOB Score (R^2 estimado): 0.2207


¿Qué quiere decir este valor? 

- R-cuadrado (R²): el oob_score del `RandomForestRegressor` es una estimación del coeficiente de determinación (R²) calculada con datos que el modelo no vio durante el entrenamiento.
- Interpretación: un R² de 0.2207 significa que el modelo, con las características (`platform`, `genre`, `rating_esrb`, `year_of_release`, `critic_score`, `user_score`) y la configuración actual (`n_estimators=100`, etc.), puede explicar aproximadamente el 22.07% de la variabilidad en las ventas totales (`total_sales`).

## Evaluación del modelo

Ahora vamos a evaluar el modelo con los datos de prueba

In [75]:
# predicciones del conjunto de prueba
predicciones = modelo.predict(X_test) 

Calculamos el RMSE que es el Error Cuadrático Medio, métrica utilizada para evaluar la precisión de los modelos de regresión. 
- Mide la diferencia promedio entre los valores que un modelo predice y los valores reales observador, de manera que penaliza más los errores grandes. 

In [76]:
# calculamos el RMSE
rmse = root_mean_squared_error(y_test, predicciones) 

# imprimimos resultados
print(f"RMSE en el conjunto de prueba: {rmse:.4f}")
print(f"Esto significa que, en promedio, nuestras predicciones se desvían ±{rmse:.2f} millones del valor real")

RMSE en el conjunto de prueba: 2.3645
Esto significa que, en promedio, nuestras predicciones se desvían ±2.36 millones del valor real


In [77]:
# comparar predicciones vs reales
df_comparacion = pd.DataFrame({'Real': y_test, 'Predicción': predicciones}).reset_index(drop=True)
print("Comparación (primeras 20 predicciones vs reales):")
display(df_comparacion.head(20))

Comparación (primeras 20 predicciones vs reales):


,Real,Predicción
0,0.13,0.102200
1,0.53,0.332500
2,0.11,0.183700
3,0.86,1.152300
4,2.11,2.177725
5,1.30,1.529500
6,0.34,0.134800
7,0.14,0.790800
8,0.11,0.264100
9,0.04,0.019900


Un error promedio de 2.37 millones es bastante significativo en comparación con las ventas de muchos juegos. Esto confirma lo que sospechábamos con el OOB Score bajo (0.22): el modelo tiene un poder predictivo limitado con las características actuales. 

No es inútil, pero tampoco es preciso.